# 1) Reading in  and Cleaning the Data

In [1]:
import os
import pandas as pd
import numpy as np
import glob
import substring
from fuzzymatcher import link_table, fuzzy_left_join
os.chdir('/Users/mariomoreno/Desktop/ML Projects/Mexico OC/data')

In [2]:
industria_cols = ['entidad', 'prod_oro_kg', 'prod_plata_kg', 'del', 'prod_plomo_t', 'prod_cobre_t', 'prod_zinc_t', 'del', 'prod_coque_t', 'del', 'prod_fierropellets_t', 'del', 'prod_azufre_t', 'del', 'prod_barita_t', 'prod_fluorita_t']
medio_cols = ['del', 'entidad', 'capital', 'altitud', 'costa_km', 'municipios', 'del']
seguridad_cols = ['entidad', 'denuncias_total', 'denuncias_salud', 'denuncias_arma', 'denuncias_bancaria', 'denuncias_leyg', 'denuncias_comms', 'del', 'denuncias_servpub', 'denuncias_fiscales', 'denuncias_ambiente', 'denuncias_asocdelictuosa', 'denuncias_patrimoniales', 'denuncias_propintelectual', 'denuncias_robocarretera', 'denuncias_otras']
carretera_cols = ['entidad', 'total_carretera_km', 'del', 'carretera_pavimentada_km', 'del', 'carretera_revestida_km', 'carretera_terraceria_kg', 'brechas_mejoradas_km', 'del']
educacion_cols = ['entidad', 'alumnos_inscritos_hs', 'hombres_inscritos_hs', 'mujeres_inscritas_hs', 'maestros_hs', 'escuelas_hs', 'del']
trabajo_cols = ['entidad', 'convenios_trabajo_no_juicio', 'conflictos_trabajo', 'emplazamientos_huelga', 'emplazamientos_huelga_solucionados', 'huelgas_estalladas', 'huelgas_locales_solucionadas', 'del']

Here I'm just simply renaming the columns to make it easier to delete unnecessary ones down the road.

In [3]:
medio_base = pd.read_excel('Medio_Ambiente.xlsx', sheetname = '1.11', header = 3, names = medio_cols)
seguridad_base = pd.read_excel('Seguridad.xlsx', sheetname = '7.2', header = 4, names = seguridad_cols)
carretera_base = pd.read_excel('Carretera.xlsx', sheetname = '14.2', header = 5, names = carretera_cols)
educacion_base = pd.read_excel('Educacion.xlsx', sheetname = '4.19', header = 4, names = educacion_cols)
industria_base = pd.read_excel('Industria.xlsx', sheetname = '11.5', header = 7, names = industria_cols)
trabajo_base = pd.read_excel('Trabajo.xlsx', sheetname = '8.15', header = 3, names = trabajo_cols)

Reading in the data, with appropriate sheetnames, headers in the right location, and appropriate column names as outlined above...though come to think of it, did not need headers if column names were changing.

Important note: the data is a number of different datasets that might have information pertinent to measuring the presence of organized crime in a Mexican state

In [4]:
base_frames = [medio_base, seguridad_base, carretera_base, educacion_base, industria_base, trabajo_base]
index_list = []
for frame in base_frames:
    frame.drop('del', axis =1, inplace=True)
    for ind, row in frame.iterrows():
        if row['entidad'] == 2010 or row['entidad'] == '2010/2011':
            print(ind)

4
543
543
408
560


Dropping all the empty columns, getting index numbers to cut frames starting at 2010 since I can't figure out how to do it otherwise

In [5]:
seguridad_base = seguridad_base.truncate(4).dropna(axis=0, how='all').reset_index(drop=True)
carretera_base = carretera_base.truncate(543).dropna(axis=0, how='all').reset_index(drop=True)
educacion_base = educacion_base.truncate(543).dropna(axis=0, how='all').reset_index(drop=True)
industria_base = industria_base.truncate(408).dropna(axis=0, how='all').reset_index(drop=True)
trabajo_base = trabajo_base.truncate(560).dropna(axis=0, how='all').reset_index(drop=True)
medio_base = medio_base.dropna(axis=0, how='all').reset_index(drop=True)

In [6]:
medio_base.fillna(value={'entidad': 'Estados Unidos Mexicanos'}, inplace=True)

,entidad,capital,altitud,costa_km,municipios
0,Estados Unidos Mexicanos,NaN,NaN,11122.0,2464.0
1,Aguascalientes,Aguascalientes,1870.0,NaN,11.0
2,Baja California,Mexicali,3.0,1493.0,5.0
3,Baja California Sur,La Paz,10.0,2131.0,5.0
4,Campeche,San Francisco de Campeche,10.0,425.0,11.0
5,Coahuila de Zaragoza,Saltillo,1700.0,NaN,38.0
6,Colima,Colima,500.0,142.0,10.0
7,Chiapas,Tuxtla Gutiérrez,640.0,266.0,124.0
8,Chihuahua,Chihuahua,1561.0,NaN,67.0
9,Ciudad de México b/,NaN,2240.0,NaN,16.0


In [7]:
base_frames = [medio_base, seguridad_base, carretera_base, educacion_base, industria_base, trabajo_base]
for frame in base_frames:
    test = []
    for ind, row in frame.iterrows():
        if type(row['entidad']) is int or row['entidad'] == '2016 P/' or row['entidad'] == '2017 P/':
            test.append((row['entidad'], ind))
        elif row['entidad'] == '2010/2011' or row['entidad'] == '2011/2012' or row['entidad'] == '2012/2013' or row['entidad'] == '2013/2014' or row['entidad'] == '2014/2015' or row['entidad'] == '2015/2016' or row['entidad'] == '2016/2017':
            test.append((row['entidad'], ind))
    print(test)

[]
[(2010, 0), (2011, 34), (2012, 69), (2013, 104), (2014, 140), (2015, 175), ('2016 P/', 211)]
[(2010, 0), (2011, 34), (2012, 68), (2013, 102), (2014, 136), (2015, 170), ('2016 P/', 204), ('2017 P/', 238)]
[('2010/2011', 0), ('2011/2012', 34), ('2013/2014', 102), ('2014/2015', 136), ('2015/2016', 170)]
[(2010, 0), (2011, 24), (2012, 48), (2013, 72), (2014, 96), (2016, 144), ('2017 P/', 168)]
[(2010, 0), (2011, 35), (2012, 70), (2013, 105), (2014, 140), (2015, 175), (2016, 210), (2017, 245)]


Ok, so after truncating from 2010 onwards, dropping all rows that were just NAs, and reseting the indices, I know have indices where each year begins. This is useful because it let's me see that I only have complete data through 2016 and it let's me start thinking about how I can create a column for year across all datasets

In [8]:
carretera_base = carretera_base[:238]
educacion_base = educacion_base[:238]
industria_base = industria_base[:168]
trabajo_base = trabajo_base[:245]

In [9]:
# Trabajo - year column
tra = trabajo_base['entidad'].tolist()
year_tra = []
for ind, ent in enumerate(tra):
    if ind >= 0 and ind < 35:
        ent = 2010
        year_tra.append(ent)
    elif ind >= 35 and ind < 70:
        ent = 2011
        year_tra.append(ent)
    elif ind >= 70 and ind < 105:
        ent = 2012
        year_tra.append(ent)
    elif ind >= 105 and ind < 140:
        ent = 2013
        year_tra.append(ent)
    elif ind >= 140 and ind < 175:
        ent = 2014
        year_tra.append(ent)
    elif ind >= 175 and ind < 210:
        ent = 2015
        year_tra.append(ent)
    else:
        ent = 2016
        year_tra.append(ent)

tcol = pd.Series(year_tra)
trabajo_base['year'] = tcol.values

# Industria - year column
inds = industria_base['entidad'].tolist()
year_ind = []
for ind, ent in enumerate(inds):
    if ind >= 0 and ind < 24:
        ent = 2010
        year_ind.append(ent)
    elif ind >= 24 and ind < 48:
        ent = 2011
        year_ind.append(ent)
    elif ind >= 48 and ind < 72:
        ent = 2012
        year_ind.append(ent)
    elif ind >= 72 and ind < 96:
        ent = 2013
        year_ind.append(ent)
    elif ind >= 96 and ind < 120:
        ent = 2014
        year_ind.append(ent)
    elif ind >= 120 and ind < 144:
        ent = 2015
        year_ind.append(ent)
    else:
        ent = 2016
        year_ind.append(ent)

icol = pd.Series(year_ind)
industria_base['year'] = icol.values

# Educacion - year column
edu = educacion_base['entidad'].tolist()
year_ed = []
for ind, ent in enumerate(edu):
    if ind >= 0 and ind < 34:
        ent = 2010
        year_ed.append(ent)
    elif ind >= 34 and ind < 68:
        ent = 2011
        year_ed.append(ent)
    elif ind >= 68 and ind < 102:
        ent = 2012
        year_ed.append(ent)
    elif ind >= 102 and ind < 136:
        ent = 2013
        year_ed.append(ent)
    elif ind >= 136 and ind < 170:
        ent = 2014
        year_ed.append(ent)
    elif ind >= 170 and ind < 204:
        ent = 2015
        year_ed.append(ent)
    else:
        ent = 2016
        year_ed.append(ent)

ecol = pd.Series(year_ed)
educacion_base['year'] = ecol.values

# Carretera - year column
car = carretera_base['entidad'].tolist()
year_car = []
for ind, ent in enumerate(car):
    if ind >= 0 and ind < 34:
        ent = 2010
        year_car.append(ent)
    elif ind >= 34 and ind < 68:
        ent = 2011
        year_car.append(ent)
    elif ind >= 68 and ind < 102:
        ent = 2012
        year_car.append(ent)
    elif ind >= 104 and ind < 136:
        ent = 2013
        year_car.append(ent)
    elif ind >= 136 and ind < 170:
        ent = 2014
        year_car.append(ent)
    elif ind >= 170 and ind < 204:
        ent = 2015
        year_car.append(ent)
    else:
        ent = 2016
        year_car.append(ent)

ccol = pd.Series(year_car)
carretera_base['year'] = ccol.values

# Seguridad - year column
seg = seguridad_base['entidad'].tolist()
year_seg = []
for ind, ent in enumerate(seg):
    if ind >= 0 and ind < 34:
        ent = 2010
        year_seg.append(ent)
    elif ind >= 34 and ind < 69:
        ent = 2011
        year_seg.append(ent)
    elif ind >= 69 and ind < 104:
        ent = 2012
        year_seg.append(ent)
    elif ind >= 104 and ind < 140:
        ent = 2013
        year_seg.append(ent)
    elif ind >= 140 and ind < 175:
        ent = 2014
        year_seg.append(ent)
    elif ind >= 175 and ind < 211:
        ent = 2015
        year_seg.append(ent)
    else:
        ent = 2016
        year_seg.append(ent)

scol = pd.Series(year_seg)
seguridad_base['year'] = scol.values

/Users/mariomoreno/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/mariomoreno/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/mariomoreno/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

With the code above, I have added the year column to each dataset with the corresponding years. It's a bit of a monster, and all of this needs to be generalized -- but it's a start

In [10]:
join = fuzzy_left_join(educacion_base, trabajo_base, ['entidad', 'year'], ['entidad', 'year'])
join.drop(['best_match_score', '__id_left', '__id_right', 'entidad_right', 'year_right'], axis = 1, inplace=True)
join2 = fuzzy_left_join(join, seguridad_base, ['entidad_left', 'year_left'], ['entidad', 'year'])
join2.drop(['best_match_score', '__id_left', '__id_right', 'entidad_left', 'year_left'], axis=1, inplace=True)
join3 = fuzzy_left_join(join2, carretera_base, ['entidad', 'year'], ['entidad', 'year'])
join3.drop(['best_match_score', '__id_left', '__id_right', 'entidad_right', 'year_right'], axis=1, inplace=True)
almost = fuzzy_left_join(join3, industria_base, ['entidad_left', 'year_left'], ['entidad', 'year'])

Pip installed fuzzymatcher and used it to match entidades and years. It was necessary because a couple of states were getting left out when just merging on entidades and years. I did this instead of coding up my own jaro-winkler function to do the matching -- worked like a charm.

In [11]:
almost.columns

Index(['best_match_score', '__id_left', '__id_right', 'alumnos_inscritos_hs',
       'hombres_inscritos_hs', 'mujeres_inscritas_hs', 'maestros_hs',
       'escuelas_hs', 'convenios_trabajo_no_juicio', 'conflictos_trabajo',
       'emplazamientos_huelga', 'emplazamientos_huelga_solucionados',
       'huelgas_estalladas', 'huelgas_locales_solucionadas', 'entidad_left',
       'denuncias_total', 'denuncias_salud', 'denuncias_arma',
       'denuncias_bancaria', 'denuncias_leyg', 'denuncias_comms',
       'denuncias_servpub', 'denuncias_fiscales', 'denuncias_ambiente',
       'denuncias_asocdelictuosa', 'denuncias_patrimoniales',
       'denuncias_propintelectual', 'denuncias_robocarretera',
       'denuncias_otras', 'year_left', 'total_carretera_km',
       'carretera_pavimentada_km', 'carretera_revestida_km',
       'carretera_terraceria_kg', 'brechas_mejoradas_km', 'entidad',
       'prod_oro_kg', 'prod_plata_kg', 'prod_plomo_t', 'prod_cobre_t',
       'prod_zinc_t', 'prod_coque_t', 'pro

In [12]:
almost.drop(['best_match_score', '__id_left', '__id_right', 'entidad', 'year'], axis=1, inplace=True)

In [13]:
almost.columns

Index(['alumnos_inscritos_hs', 'hombres_inscritos_hs', 'mujeres_inscritas_hs',
       'maestros_hs', 'escuelas_hs', 'convenios_trabajo_no_juicio',
       'conflictos_trabajo', 'emplazamientos_huelga',
       'emplazamientos_huelga_solucionados', 'huelgas_estalladas',
       'huelgas_locales_solucionadas', 'entidad_left', 'denuncias_total',
       'denuncias_salud', 'denuncias_arma', 'denuncias_bancaria',
       'denuncias_leyg', 'denuncias_comms', 'denuncias_servpub',
       'denuncias_fiscales', 'denuncias_ambiente', 'denuncias_asocdelictuosa',
       'denuncias_patrimoniales', 'denuncias_propintelectual',
       'denuncias_robocarretera', 'denuncias_otras', 'year_left',
       'total_carretera_km', 'carretera_pavimentada_km',
       'carretera_revestida_km', 'carretera_terraceria_kg',
       'brechas_mejoradas_km', 'prod_oro_kg', 'prod_plata_kg', 'prod_plomo_t',
       'prod_cobre_t', 'prod_zinc_t', 'prod_coque_t', 'prod_fierropellets_t',
       'prod_azufre_t', 'prod_barita_t', 'pro

In [14]:
almost.reset_index(drop=True, inplace=True)

Doing a lot of cleaning of this final dataframe, including removing the extra columns put in there by fuzzymatcher and resetting the index. Next, I'll try to remove the year rows, add the coastal information, and set a multi-index for entidad and year.

In [15]:
to_drop = []
for index, row in almost.iterrows():
    if type(row['entidad_left']) is int or row['entidad_left'] == '2016 P/':
        to_drop.append(index)

In [16]:
almost.drop(almost.index[to_drop], inplace=True)

In [17]:
almost['entidad_left'].value_counts()

Guanajuato                         7
Nayarit                            7
Coahuila de Zaragoza               7
Durango                            7
Querétaro                          7
Aguascalientes                     7
Baja California Sur                7
Morelos                            7
Baja California                    7
Zacatecas                          7
Michoacán de Ocampo                7
Tamaulipas                         7
Sinaloa                            7
Chiapas                            7
Estados Unidos Mexicanos           7
Ciudad de México                   7
Puebla                             7
San Luis Potosí                    7
México                             7
Quintana Roo                       7
Campeche                           7
Tlaxcala                           7
Nuevo León                         7
Yucatán                            7
Chihuahua                          7
Colima                             7
Guerrero                           7
V

In [18]:
final = fuzzy_left_join(almost, medio_base, ['entidad_left'], ['entidad'])

In [19]:
final['entidad'].value_counts()

Nuevo León                          7
Michoacán de Ocampo                 7
Guerrero                            7
Tamaulipas                          7
Puebla                              7
Querétaro                           7
Campeche                            7
Tabasco                             7
Baja California Sur                 7
Yucatán                             7
México                              7
Veracruz de Ignacio de la Llave     7
Morelos                             7
Jalisco                             7
Guanajuato                          7
Estados Unidos Mexicanos            7
Coahuila de Zaragoza                7
San Luis Potosí                     7
Tlaxcala                            7
Baja California                     7
Colima                              7
Quintana Roo                        7
Aguascalientes                      7
Sinaloa                             7
Nayarit                             7
Chihuahua                           7
Oaxaca      

Now that we have a dataframe that inclues all the values I want, it's critical to note that the denuncias and education variables are all in gross numbers. This is problematic since larger states will naturally have larger amounts of people requesting services. I need to add population to the columns. 

Found a new set of data, but it's an excel file for every state. Will have to use glob

In [20]:
population_xlsx = glob.glob('../data/Estimaciones*.xlsx')

In [21]:
def clean_glob(df):
    entidad = substring.substringByChar(df.columns.values[0], endChar=':')
    df.iloc[[3],[0]] = entidad
    df.columns = ['entidad', '2010', '2011', '2012', '2013', '2014', '2015', '2016']
    df.dropna(axis=0, how='all', inplace=True)
    new_df = df.iloc[1]
    return new_df

In [22]:
pop_data = pd.DataFrame()
for state in population_xlsx:
    df = pd.read_excel(state, sheetname='Indicadores demográficos', usecols=[0,1,2,3,4,5,6,7])
    new_df = clean_glob(df)
    pop_data = pop_data.append(new_df, ignore_index=True)

In [23]:
pop_data

,2010,2011,2012,2013,2014,2015,2016,entidad
0,2.851334e+06,2.878525e+06,2.905750e+06,2.932313e+06,2.958691e+06,2.984571e+06,3.009952e+06,Sinaloa:
1,1.557168e+07,1.584556e+07,1.610648e+07,1.636421e+07,1.661893e+07,1.687039e+07,1.711852e+07,México:
2,8.944599e+06,8.928400e+06,8.911665e+06,8.893742e+06,8.874724e+06,8.854600e+06,8.833416e+06,Distrito Federal:
3,3.444264e+06,3.473454e+06,3.499507e+06,3.523858e+06,3.546710e+06,3.568139e+06,3.588255e+06,Guerrero:
4,1.509019e+06,1.522741e+06,1.536674e+06,1.550179e+06,1.563324e+06,1.576068e+06,1.588418e+06,Zacatecas:
5,2.782013e+06,2.818077e+06,2.854334e+06,2.890108e+06,2.925594e+06,2.960681e+06,2.995374e+06,Coahuila:
6,1.848191e+06,1.881105e+06,1.912803e+06,1.943889e+06,1.974436e+06,2.004472e+06,2.034030e+06,Querétaro:
7,1.186143e+06,1.206291e+06,1.224637e+06,1.242734e+06,1.260628e+06,1.278308e+06,1.295781e+06,Tlaxcala:
8,1.350945e+06,1.395357e+06,1.440115e+06,1.484960e+06,1.529877e+06,1.574824e+06,1.619762e+06,Quintana Roo:
9,1.803340e+06,1.827187e+06,1.850812e+06,1.874188e+06,1.897393e+06,1.920350e+06,1.943044e+06,Morelos:


Got the population data, but will now need to melt it in order to match the rest of the data at hand

In [24]:
melted_pop = pop_data.melt(id_vars='entidad')

In [25]:
melted_pop.columns = ['entidad', 'year', 'pop']
melted_pop['pop'] = melted_pop['pop'].astype(int)
melted_pop['entidad'] = melted_pop['entidad'].replace({'Distrito Federal:':'Ciudad de México'})
melted_pop

,entidad,year,pop
0,Sinaloa:,2010,2851333
1,México:,2010,15571679
2,Ciudad de México,2010,8944598
3,Guerrero:,2010,3444264
4,Zacatecas:,2010,1509019
5,Coahuila:,2010,2782012
6,Querétaro:,2010,1848191
7,Tlaxcala:,2010,1186142
8,Quintana Roo:,2010,1350944
9,Morelos:,2010,1803340


In [26]:
final.drop(['best_match_score', '__id_left', '__id_right', 'entidad_left', 'capital'], axis=1, inplace=True)
final.rename(columns={'year_left':'year'}, inplace=True)

In [27]:
this = fuzzy_left_join(final, melted_pop, ['entidad', 'year'], ['entidad', 'year'])

In [28]:
this['entidad_left'].value_counts()

Nuevo León                          7
Michoacán de Ocampo                 7
Guerrero                            7
Tamaulipas                          7
Puebla                              7
Querétaro                           7
Campeche                            7
Tabasco                             7
Baja California Sur                 7
Yucatán                             7
México                              7
Veracruz de Ignacio de la Llave     7
Morelos                             7
Jalisco                             7
Guanajuato                          7
Estados Unidos Mexicanos            7
Coahuila de Zaragoza                7
San Luis Potosí                     7
Tlaxcala                            7
Baja California                     7
Colima                              7
Quintana Roo                        7
Aguascalientes                      7
Sinaloa                             7
Nayarit                             7
Chihuahua                           7
Oaxaca      

In [29]:
this.drop(['best_match_score', '__id_left', '__id_right', 'entidad_right', 'year_right'], axis=1, inplace=True)
this.rename(columns={'year_left':'year', 'entidad_left':'entidad'}, inplace=True)

In [30]:
final_df = this[this.entidad != 'Estados Unidos Mexicanos']

In [31]:
final_df.set_index(['entidad', 'year'], inplace=True)

In [32]:
final_df

,,alumnos_inscritos_hs,hombres_inscritos_hs,mujeres_inscritas_hs,maestros_hs,escuelas_hs,convenios_trabajo_no_juicio,conflictos_trabajo,emplazamientos_huelga,emplazamientos_huelga_solucionados,huelgas_estalladas,...,prod_zinc_t,prod_coque_t,prod_fierropellets_t,prod_azufre_t,prod_barita_t,prod_fluorita_t,altitud,costa_km,municipios,pop
entidad,year,,,,,,,,,,,,,,,,,,,,,
Aguascalientes,2010,40129,19250,20879,2727.0,144,2698.0,3239.0,325.0,195.0,0.0,...,0.0,0.0,3883423.0,0.0,0.0,0.0,1870.0,NaN,11.0,1195787
Baja California,2010,107624,53692,53932,6192.0,271,2342.0,9353.0,3241.0,1010.0,32.0,...,0.0,0.0,3883423.0,0.0,0.0,0.0,3.0,1493.0,5.0,3224843
Baja California Sur,2010,23247,11780,11467,1588.0,76,1255.0,1660.0,296.0,127.0,1.0,...,0.0,0.0,3883423.0,0.0,0.0,0.0,10.0,2131.0,5.0,649616
Campeche,2010,28350,14349,14001,1975.0,106,3373.0,1179.0,144.0,10.0,0.0,...,0.0,0.0,3883423.0,0.0,0.0,0.0,10.0,425.0,11.0,836747
Coahuila de Zaragoza,2010,82553,41397,41156,6219.0,330,23331.0,7867.0,545.0,102.0,0.0,...,0.0,1648709.0,2567865.0,0.0,22161.0,121833.0,1700.0,NaN,38.0,2782012
Colima,2010,23654,11567,12087,1528.0,91,1962.0,935.0,233.0,61.0,1.0,...,0.0,0.0,3883423.0,0.0,0.0,0.0,500.0,142.0,10.0,658909
Chiapas,2010,183881,95643,88238,9449.0,704,4008.0,1584.0,192.0,0.0,0.0,...,0.0,0.0,0.0,236841.0,0.0,0.0,640.0,266.0,124.0,4903754
Chihuahua,2010,114630,55629,59001,6405.0,457,10818.0,9810.0,246.0,36.0,0.0,...,133734.0,0.0,0.0,0.0,0.0,0.0,1561.0,NaN,67.0,3525273
Ciudad de México b/,2010,387690,195874,191816,31028.0,591,18035.0,28681.0,4561.0,1808.0,22.0,...,35562.0,0.0,0.0,0.0,0.0,0.0,2240.0,NaN,16.0,8944598


Finished cleaning up this dataframe and indexing it by entidad and year. I'm exporting the final excel file and opening up a new notebook for finding and imputing missing variables.

In [37]:
writer = pd.ExcelWriter('Mexico_Final.xlsx')
final_df.to_excel(writer)
writer.save()